## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

### Load a WireCutter object with our QuantumCircuit and find a cutting scheme that meets our criteria

In [3]:
cutter = WireCutter(qc)
cuts = cutter.cut_automatic(
    max_subcircuit_width=3,
    max_subcircuit_cuts=10,
    max_subcircuit_size=12,
    max_cuts=10,
    num_subcircuits=[2],
)

Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 23 rows and 5 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 22 substitutions.
Reduced MIP has 47 rows, 24 columns, and 142 nonzeros.
Reduced MIP has 15 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.36 ticks)
Found incumbent of value 1.000000 after 0.01 sec. (0.51 ticks)
Probing fixed 15 vars, tightened 22 bounds.
Probing time = 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 47 rows and 24 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.56 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0

### Define a constant shots function, evaluate our circuit, and verify the results

In [4]:
def num_shots_fn(circuit):
    return 1024


subcircuit_instance_probabilities = cutter.evaluate(cuts, mode="sv", num_shots_fn=num_shots_fn,)
ordered_probabilities = cutter.recompose(subcircuit_instance_probabilities, cuts, num_threads=4)
metrics = cutter.verify(ordered_probabilities)
print(metrics)

(array([0.22767293, 0.04553459, 0.0390625 , 0.0078125 , 0.04553459,
       0.04553459, 0.0078125 , 0.0078125 , 0.0078125 , 0.0078125 ,
       0.00134041, 0.00134041, 0.0390625 , 0.0078125 , 0.00670207,
       0.00134041, 0.22767293, 0.04553459, 0.0390625 , 0.0078125 ,
       0.04553459, 0.04553459, 0.0078125 , 0.0078125 , 0.0078125 ,
       0.0078125 , 0.00134041, 0.00134041, 0.0390625 , 0.0078125 ,
       0.00670207, 0.00134041]), {'nearest': {'chi2': 0, 'Mean Squared Error': 3.699354190434665e-34, 'Mean Absolute Percentage Error': 7.342176075473582e-14, 'Cross Entropy': 2.5996810883678423, 'HOP': 0.9004283905932733}, 'naive': {'chi2': 0, 'Mean Squared Error': 1.3291917065602401e-33, 'Mean Absolute Percentage Error': 7.326395851265026e-14, 'Cross Entropy': 2.5996810883678423, 'HOP': 0.9004283905932738}})
